In [ ]:
import zipfile

In [ ]:
fantasy_zip = zipfile.ZipFile('./Stock_Dataset(2017_07_06).zip')
fantasy_zip.extractall('./data')

In [93]:
import pandas as pd
from pandas import Series, DataFrame
from sklearn import preprocessing
import numpy as np
import tensorflow as tf
import math

In [2]:
!pip install xlrd

mxnet-cu80 1.1.0 has requirement numpy<=1.13.3, but you'll have numpy 1.14.3 which is incompatible.


In [33]:
!pip install openpyxl

    100% |████████████████████████████████| 174kB 2.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for openpyxl ... done
  Stored in directory: /home/U2017042/.cache/pip/wheels/d3/75/83/415a5bbed8366f66a87f7bd0fc28f7df6e13982960f3e5a7ab
  Running setup.py bdist_wheel for et-xmlfile ... done
  Stored in directory: /home/U2017042/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile
mxnet-cu80 1.1.0 has requirement numpy<=1.13.3, but you'll have numpy 1.14.3 which is incompatible.


In [3]:
corporations = pd.read_excel('./corporations.xlsx')

In [61]:
stock_corps = corporations.query("상장일<'2005-01-01'  ")[['회사명', '종목코드']]
print(stock_corps)

           회사명   종목코드
0          BYC   1460
1       CJ CGV  79160
2       DB손해보험   5830
4        DSR제강  69730
6      KC그린홀딩스   9440
7     KISCO홀딩스   1940
9        MH에탄올  23150
10        NICE  34310
11      S&T홀딩스  36530
12       SK텔레콤  17670
14        계양전기  12200
15     골든브릿지증권   1290
16        기업은행  24110
17       깨끗한나라   4540
18        남광토건   1260
19        남해화학  25860
20         다스코  58730
21       대구백화점   6370
22        대신증권   3540
23        대한제분   1130
24        대한항공   3490
25        대한해운   5880
26       대호에이엘  69460
27        덕양산업  24900
29         디씨엠  24090
31        롯데지주   4990
32       롯데케미칼  11170
33         마니커  27740
34     메리츠종금증권   8560
35         모토닉   9680
...        ...    ...
2111      유아이디  69330
2112      유진기업  23410
2116       이젠텍  33600
2117     이테크건설  16250
2118     인선이엔티  60150
2121     전파기지국  65530
2123    제이콘텐트리  36420
2124      중앙백신  72020
2127       지엔코  65060
2128      청보산업  13720
2129       초록뱀  47820
2136       코미팜  41960
2138      큐캐피탈  16600
2139   크린앤

In [213]:
# 상세 데이터를 가져온다.
def get_stock_datail(comp_code) :
    code = format(comp_code, "06d");
    return pd.read_csv('./data/' + code + '.csv')

# matrix 데이터로 변경한다.
def to_ndarray(cols_data) :
    if isinstance(cols_data, Series):
        return np.reshape(list(cols_data), (-1,1))
    elif isinstance(cols_data, DataFrame):
        return cols_data.as_matrix()

# 컬럼을 스케일링을 시킨다.
def get_scaled_cols(data, column_name) :
    scale_data = to_ndarray(data[column_name])
    scaler = preprocessing.MinMaxScaler()
    return scaler.fit_transform(scale_data);

# 데이터를 스케일링 시킨다.
def get_scaled_data(data) :
    scaled_data = data.copy()
    scaled_data['Close'] = get_scaled_cols(scaled_data, 'Close')
    scaled_data['Open'] = get_scaled_cols(scaled_data, 'Open')
    scaled_data['High'] = get_scaled_cols(scaled_data, 'High')
    scaled_data['Low'] = get_scaled_cols(scaled_data, 'Low')
    scaled_data['Volume'] = get_scaled_cols(scaled_data, 'Volume')
    return scaled_data;

# RNN을 위한 데이터로 만든다. 
def get_dataXY(data, train_params) :
    x = to_ndarray(data[['Open', 'High', 'Low', 'Volume', 'Close']])
    y = to_ndarray(data['Close'])
    
    dataX = []
    dataY = []
    seq_length = train_params['seq_length']
    for i in range(0, len(y) - seq_length):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length]  # Next close price
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return dataX, dataY

# train 및 test 데이터로 나눈다.
def split_train_test(dataX, dataY, train_params) :
    invest_count = train_params['invest_count']
    data_count = len(dataY);
    train_size = int(data_count * train_params['train_percent'] / 100)
    train_last = data_count-invest_count;
    
    trainX = np.array(dataX[0:train_size])
    testX = np.array(dataX[train_size:train_last])
    #investX = np.array(dataX[train_last:data_count])
    
    trainY = np.array(dataY[0:train_size])
    testY = np.array(dataY[train_size:train_last])
    #investY = np.array(dataY[train_last:data_count])
    return {
        'trainX': trainX, 'trainY': trainY, 
        'testX': testX, 'testY': testY,
        #'investX': investX,'investY': investY
    }

def split_train_test_for_invest(dataX, dataY, train_params, index, data) :
    invest_count = train_params['invest_count']
    seq_length = train_params['seq_length']
    data_count = len(dataY);
    train_size = data_count - invest_count + index
    real_index = train_size + seq_length - 1
    
    trainX = np.array(dataX[0:train_size])
    testX = np.array(dataX[train_size:train_size+1])
    realClose = data['Close'][real_index:real_index+1].values[0]
    
    trainY = np.array(dataY[0:train_size])
    testY = np.array(dataY[train_size:train_size+1])
    
    return {
        'trainX': trainX, 'trainY': trainY, 
        'testX': testX, 'testY': testY,
        'realClose': realClose
    }

# train, test데이터로 만든다.
def get_train_test(data, train_params) :
    scaled_data = get_scaled_data(data)
    dataX, dataY = get_dataXY(scaled_data, train_params)
    return split_train_test(dataX, dataY, train_params)

def get_train_test_for_invest(data, train_params, index) :
    scaled_data = get_scaled_data(data)
    dataX, dataY = get_dataXY(scaled_data, train_params)
    return split_train_test_for_invest(dataX, dataY, train_params, index, data)

# 텐스플로우 변수관계 그래프롤 그린다.
def draw_graph(train_params) :
    
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, [None, train_params['seq_length'], 
                                    train_params['data_dim']])
    Y = tf.placeholder(tf.float32, [None, 1])
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=train_params['hidden_dim'], 
                                        state_is_tuple=True, 
                                        activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32 )
    Y_pred = tf.contrib.layers.fully_connected(
        outputs[:, -1], train_params['output_dim'], activation_fn=None)  # We use the last cell's output

    # cost/loss
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    # optimizer
    optimizer = tf.train.AdamOptimizer(train_params['learning_rate'])
    train = optimizer.minimize(loss)

    # RMSE
    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])
    rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))
    return {
        'X': X,
        'Y': Y,
        'train': train,
        'loss' : loss,
        'Y_pred': Y_pred,
        'targets': targets,
        'rmse' : rmse,
        'predictions': predictions
    }

# 학습을 시킨다.
def let_training(data_params, train_params, graph_params, comp_code) :
    X = graph_params['X']
    Y = graph_params['Y']
    train = graph_params['train']
    loss = graph_params['loss']
    trainX = data_params['trainX']
    trainY = data_params['trainY']
    testX = data_params['testX']
    testY = data_params['testY']
    Y_pred = graph_params['Y_pred']
    targets = graph_params['targets']
    rmse = graph_params['rmse']
    predictions = graph_params['predictions']
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(train_params['iterations']):
            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
        
        test_predict = sess.run(Y_pred, feed_dict={X: testX})
        rmse_val = sess.run(rmse, feed_dict={targets: testY, predictions: test_predict})
        #print("RMSE: {}".format(rmse_val))
        #tf.add_to_collection("X", X)
        #tf.add_to_collection("Y_pred", Y_pred)
        #saver.save(sess, "./sessions/" + str(comp_code) + ".ckpt")
        return test_predict, rmse_val 

# 그래프를 그리고 학습을 시킨다.    
def let_leaning(data_params, train_params, comp_code):
    graph_params = draw_graph(train_params)
    return let_training(data_params, train_params, graph_params, comp_code)

def to_dataFrame(data, columns) :
    return pd.DataFrame(data, columns=columns)

# excel로 저장한다.
def save_excel(df_data, file_name):
    writer = pd.ExcelWriter(file_name)
    df_data.to_excel(writer,'Sheet1', index=False)
    writer.save()
    
def let_invest_money(invest_predict, data_params, train_params, now_money, now_stock_cnt) :
    seq_length = train_params['seq_length']
    data_dim = train_params['data_dim']
    pie_percent = train_params['pie_percent']
    invest_min_percent = train_params['invest_min_percent']
    testX = data_params['testX']
    now_close = data_params['realClose']
    
    now_scaled_close = testX[len(testX)-1][seq_length-1][data_dim-1]
    pred_scaled_close = invest_predict[0][0]
    pie = now_close * pie_percent/100
    ratio = (pred_scaled_close - now_scaled_close) /now_scaled_close * 100
    
    if ratio > invest_min_percent :
        cnt = math.floor(now_money/now_close)
        if cnt > 0 :
            now_money -= (now_close + pie) * cnt
            now_stock_cnt += cnt
    elif ratio < -invest_min_percent :
        if now_stock_cnt > 0 :
            now_money += to_money(data_params, now_stock_cnt)
            now_stock_cnt = 0
    #print(now_money, now_stock_cnt, now_scaled_close, pred_scaled_close, data_params['testY'])
    return now_money, now_stock_cnt

def to_money(data_params, now_stock_cnt) :
    money = 0
    if now_stock_cnt > 0 :
        now_close = data_params['realClose']
        pie_percent = train_params['pie_percent']
        tax_percent = train_params['tax_percent']
        
        pie = now_close * pie_percent/100
        tax = now_close * tax_percent/100
        money = (now_close - (pie + tax)) * now_stock_cnt
    return money
    

def let_invest(row, train_params):
    comp_code = row['code']
    invest_count = train_params['invest_count']
    invest_money = train_params['invest_money']
        
    data = get_stock_datail(comp_code)
    now_money = invest_money
    now_stock_cnt = 0
    
    for i in range(0, invest_count) :
        data_params = get_train_test_for_invest(data, train_params, i)
        predict, _ = let_leaning(data_params, train_params, comp_code)
        now_money, now_stock_cnt = let_invest_money(predict, data_params, train_params, now_money, now_stock_cnt)
    now_money += to_money(data_params, now_stock_cnt)
    #print(now_money)
    return now_money

In [ ]:
# train Parameters
train_params = {
    'seq_length' : 10,
    'data_dim' : 5,
    'hidden_dim' : 5,
    'output_dim' : 1,
    'learning_rate' : 0.001,
    'iterations' : 1000,
    'train_percent' : 70,
    'invest_count' : 50,
    'invest_money' : 1000000,
    'pie_percent' : 0.015,
    'tax_percent' : 0.5,
    'invest_min_percent' : 2
};


In [59]:
# 주식 종목들을 가져와서 학습을 시킨다.
comp_rmses = []
for idx, row in stock_corps.iterrows():
    comp_code = row['종목코드']
    data = get_stock_datail(comp_code)
    data_params = get_train_test(data, train_params)
    _, rmse_val = let_leaning(data_params, train_params, comp_code)
    if idx == 0 :
        print('종목코드', '회사명', 'RMSE')
    print(comp_code, row['회사명'], rmse_val)
    comp_rmses.append([comp_code, row['회사명'], rmse_val])


/opt/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


No 종목코드 회사명 RMSE
0 1460 BYC 0.041651376
1 79160 CJ CGV 0.20499137
2 5830 DB손해보험 0.06703028
4 69730 DSR제강 0.03871913
6 9440 KC그린홀딩스 0.0077934014
7 1940 KISCO홀딩스 0.014083393
9 23150 MH에탄올 0.006528043
10 34310 NICE 0.027320897
11 36530 S&T홀딩스 0.009757335
12 17670 SK텔레콤 0.0610651
14 12200 계양전기 0.029732715
15 1290 골든브릿지증권 0.005604596
16 24110 기업은행 0.025236918
17 4540 깨끗한나라 0.032585476
18 1260 남광토건 0.013392938
19 25860 남해화학 0.009065369
20 58730 다스코 0.027689153
21 6370 대구백화점 0.02381659
22 3540 대신증권 0.009711457
23 1130 대한제분 0.028460281
24 3490 대한항공 0.018804533
25 5880 대한해운 0.0041768025
26 69460 대호에이엘 0.024684392
27 24900 덕양산업 0.0835019
29 24090 디씨엠 0.036951892
31 4990 롯데지주 0.108773805
32 11170 롯데케미칼 0.01957977
33 27740 마니커 0.0054556783
34 8560 메리츠종금증권 0.011123519
35 9680 모토닉 0.004263349
36 2760 보락 0.02040928
37 3850 보령제약 0.041210197
38 79660 사조해표 0.03295426
39 2170 삼양통상 0.036038313
42 11230 삼화전자공업 0.014275532
43 41650 상신브레이크 0.028503746
46 58650 세아홀딩스 0.025870053
47 33530 세종공업 0.026650203
49 4

596 28050 삼성엔지니어링 0.007496098
597 5930 삼성전자 0.033600375
598 3230 삼양식품 0.030216353
599 4380 삼익THK 0.02906015
600 520 삼일제약 0.0038252068
601 4690 삼천리 0.01711876
602 1820 삼화콘덴서공업 0.018504238
603 390 삼화페인트공업 0.10642492
604 8490 서흥 0.023167927
605 5980 성지건설 0.015214224
606 1430 세아베스틸 0.021044042
608 3620 쌍용자동차 0.016994918
609 8700 아남전자 0.011796864
610 2790 아모레퍼시픽그룹 0.07582925
611 2310 아세아제지 0.038866792
613 670 영풍 0.01491621
614 6740 영풍제지 0.060515065
615 15360 예스코홀딩스 0.023190731
616 16880 웅진 0.018311847
617 11690 유양디앤유 0.027705817
618 220 유유제약 0.010458915
619 100 유한양행 0.024279954
620 8730 율촌화학 0.037560742
621 230 일동홀딩스 0.051549885
625 3780 진양산업 0.03804839
626 11000 진원생명과학 0.0048036166
627 33250 체시스 0.033006582
631 4870 티웨이홀딩스 0.043249313
633 47050 포스코대우 0.0251994
635 2960 한국쉘석유 0.031609304
636 6200 한국전자홀딩스 0.023841122
639 9180 한솔로지스틱스 0.10857762
640 18880 한온시스템 0.081515655
641 2220 한일철강 0.018755801
643 2320 한진 0.027152155
645 880 한화 0.014630803
648 720 현대건설 0.021252856
649 5440 현대그린푸드 0.01364

1231 10620 현대미포조선 0.012642228
1232 1500 현대차투자증권 0.009663823
1234 68790 DMS 0.016667685
1238 46440 KG모빌리언스 0.037555017
1241 36540 SFA반도체 0.043343157
1243 57500 SKC 솔믹스 0.020761052
1244 48550 SM C&C 0.015212199
1246 32540 TJ미디어 0.02388939
1249 39240 경남스틸 0.025622718
1250 11040 경동제약 0.014918474
1252 51490 나라엠앤디 0.039383795
1253 33640 네패스 0.016517617
1257 48470 대동스틸 0.028517507
1258 7720 대명코퍼레이션 0.0033771829
1259 25440 대성엘텍 0.01552001
1261 67080 대화제약 0.03455532
1265 75970 동국알앤에스 0.032488767
1268 32960 동일기연 0.03160806
1269 5290 동진쎄미켐 0.029522397
1271 33130 디지틀조선 0.05200492
1274 42500 링네트 0.038841996
1278 44480 바이오제네틱스 0.05609868
1280 36620 버추얼텍 0.03152697
1285 32750 삼진 0.009465603
1286 42940 상지카일룸 0.064663276
1287 27040 서울전자통신 0.037075892
1289 65710 서호전기 0.02625993
1290 3100 선광 0.030082047
1291 45300 성우테크론 0.029831909
1292 53060 세동 0.02920654
1298 52710 아모텍 0.036018115
1304 73070 에스모 0.02474639
1308 69510 에스텍 0.024167601
1309 41910 에스텍파마 0.034720004
1310 52020 에스티큐브 0.01757983
1311 58610 에스

1859 37330 인지디스플레 0.015023273
1861 17250 인터엠 0.03199689
1862 35080 인터파크홀딩스 0.029659087
1863 49550 잉크테크 0.01354999
1866 40420 정상제이엘에스 0.014071432
1867 33100 제룡전기 0.033868037
1868 58420 제이웨이 0.087310545
1869 23440 제일제강 0.054955237
1870 33340 좋은사람들 0.03910307
1879 42040 케이피엠테크 0.0388483
1881 46070 코다코 0.014827959
1882 47770 코데즈컴바인 0.053104866
1884 45970 코아시아홀딩스 0.020227352
1885 52330 코텍 0.030688573
1886 40350 큐로컴 0.018323477
1890 54780 키이스트 0.008407213
1899 43090 팜스웰바이오 0.017380238
1900 9520 포스코엠텍 0.009883697
1901 35200 프럼파스트 0.032756582
1908 23760 한국캐피탈 0.0018347324
1909 21650 한국큐빅 0.024379205
1910 37230 한국팩키지 0.009569718
1911 47080 한빛소프트 0.03245009
1912 70590 한솔인티큐브 0.035433915
1916 8800 행남사 0.11007327
1920 28080 휴맥스홀딩스 0.008472103
1921 65510 휴비츠 0.027892012
1933 11150 CJ씨푸드 0.027036514
1934 7700 F&F 0.032266572
1935 1390 KG케미칼 0.02561786
1937 51910 LG화학 0.018486947
1938 6260 LS 0.012335405
1939 3570 S&T중공업 0.016944734
1940 1380 SG충방 0.02326422
1941 77970 STX엔진 0.04100911
1942 500 가온전선 

In [ ]:
# 데이터를 정렬하고 저장한다.
df_comp_rmses = pd.DataFrame(comp_rmses, columns=['code', 'name', 'RMSE'])    
df_comp_rmses = df_comp_rmses.sort_values('RMSE')
save_excel(df_comp_rmses, 'comp_rmses.xlsx')

In [221]:
# 에러률이 적은 상위 100개만 가져와서 모의투자를 실행한다.
invest_results = []
df_tops = df_comp_rmses[0:100]
for index, row in df_tops.iterrows():
    now_money = let_invest(row, train_params)
    invest_results.append([comp_code, row['회사명'], mow_money])
    
df_invest_results = pd.DataFrame(invest_results, columns=['code', 'name', 'money'])    
save_excel(df_invest_results, 'invest_result.xlsx')    

/opt/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


1000000 0 0.0024355842994255045 0.0024489046 [[0.00249751]]
1000000 0 0.0024975059341566614 0.0022106403 [[0.00244246]]
2500.3974999999627 327 0.0024424644810623003 0.0025370163 [[0.00249751]]
2500.3974999999627 327 0.0024975059341566614 0.0024974097 [[0.00251127]]
1010979.8424999999 0 0.002511266297430252 0.0020756945 [[0.00251127]]
228.2524999998277 326 0.002511266297430252 0.0027078537 [[0.00253191]]
228.2524999998277 326 0.0025319068423406378 0.0024911887 [[0.00254567]]
228.2524999998277 326 0.0025456672056142285 0.0027508547 [[0.00253879]]
228.2524999998277 326 0.0025387870239774327 0.0025264234 [[0.00253879]]
1012110.0844999998 0 0.0025387870239774327 0.0016155664 [[0.00251127]]
1012110.0844999998 0 0.002511266297430252 0.0018797982 [[0.00251127]]
1012110.0844999998 0 0.002511266297430252 0.0021833754 [[0.00249751]]
1528.5199999997858 327 0.0024975059341566614 0.0034388062 [[0.00249063]]
1005128.2257499999 0 0.0024906257525198665 0.0022366962 [[0.00249063]]
2352.8319999998203 325

KeyboardInterrupt: 